# Image Classification

In [ ]:
# importing libraries
import warnings
warnings.simplefilter('ignore')

from zipfile import ZipFile
import os
import cv2 
import math
import random
import numpy as np
import pandas as pd
import seaborn as sb
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from keras.utils import to_categorical
from tqdm.notebook import tqdm 
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
import shutil

## Data Preparation

In [ ]:
num_samples = 500
folder_path = r"anike\animal classification"
data_path = os.path.join(folder_path, 'Data')
os.makedirs(data_path, exist_ok=True)

def unzip_data():
    for root, _, files in os.walk(folder_path):
        for file in files:
            path = os.path.join(root, file) 
            if path.endswith(".zip"):
                with ZipFile(path, 'r') as zf:
                    zf.extractall(path=data_path)
    print("Unzip Data Completed")

unzip_data()

Unzip Data Completed


In [ ]:

def build_data():
    data = []
    for folder in os.listdir(data_path):
        if folder.startswith("train_"):
            target = folder.split("_")[-1]
            folder_path = os.path.join(data_path, folder)
            files_class = []
            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpeg") and len(files_class) < num_samples:
                    file_path = os.path.join(folder_path, file_name)
                    files_class.append(file_path)
                    data.append({'image': file_name, 'filepath': file_path, 'target': target})
    return pd.DataFrame(data)

data = build_data()
data = data.sample(frac=1).reset_index(drop=True)

data

,image,filepath,target
0,ASG0014jz1_1.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants
1,ASG001e0h4_1.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras
2,ASG0014jzl_1.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants
3,ASG0014jxg_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants
4,ASG0014iqp_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants
...,...,...,...
995,ASG001e0qj_2.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras
996,ASG001e0nk_2.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras
997,ASG001e0mw_0.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras
998,ASG0014k67_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants


In [ ]:


# Convert string labels to numerical labels
label_encoder = LabelEncoder()
data['class'] = label_encoder.fit_transform(data['target'])
data

,image,filepath,target,class
0,ASG0014jz1_1.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants,0
1,ASG001e0h4_1.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras,1
2,ASG0014jzl_1.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants,0
3,ASG0014jxg_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants,0
4,ASG0014iqp_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants,0
...,...,...,...,...
995,ASG001e0qj_2.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras,1
996,ASG001e0nk_2.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras,1
997,ASG001e0mw_0.jpeg,C:\Users\anike\animal classification\Data\trai...,zebras,1
998,ASG0014k67_0.jpeg,C:\Users\anike\animal classification\Data\trai...,elephants,0


## Model Building

In [ ]:
class_samples = data.groupby('target').head(num_samples)
train_size = int(0.8 * len(class_samples))
train_dataset = class_samples[:train_size]
val_dataset = class_samples[train_size:]

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 800
Validation dataset size: 200


In [ ]:


train_images_folder = os.path.join(data_path, 'train_images')
val_images_folder = os.path.join(data_path, 'val_images')
os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(val_images_folder, exist_ok=True)

# Save images into folders
def save_images(dataset, target_folder):
    for index, row in dataset.iterrows():
        image_path = row['filepath']
        target_class = row['target']
        class_folder = os.path.join(target_folder, target_class)
        os.makedirs(class_folder, exist_ok=True)
        shutil.copy(image_path, class_folder)

# Save training images
save_images(train_dataset, train_images_folder)

# Save validation images
save_images(val_dataset, val_images_folder)

# Save CSV files
train_csv_path=os.path.join(data_path,'train_dataset.csv')
val_csv_path=os.path.join(data_path,'val_dataset.csv')
train_dataset.to_csv(train_csv_path, index=False)
val_dataset.to_csv(val_csv_path, index=False)

print("Images and CSV files saved successfully.")


Images and CSV files saved successfully.


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 1])
#         print(img_name)
        image = Image.open(img_name)
        label = self.data_frame.iloc[idx, 3]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and dataloaders
train_dataset = CustomDataset(csv_file=train_csv_path, root_dir=train_images_folder, transform=transform)
val_dataset = CustomDataset(csv_file=val_csv_path, root_dir=val_images_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



In [ ]:
num_classes=len(np.unique(data['target']))
num_classes

2

In [ ]:
class SimpleModel(nn.Module):
    def __init__(self, num_classes, conv1_kernel_size=5,dropout=0.3):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=conv1_kernel_size, stride=1, padding=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=conv1_kernel_size, stride=1, padding=2)
        self.fc1 = nn.Linear(16 * 56 * 56, 128)  
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))  
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x)) 
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 16 * 56 * 56)
        x = self.dropout(x)
        x = F.sigmoid(self.fc1(x)) 
        x = self.fc2(x)
        return x

## Training model without hyperparameter tuning

In [ ]:
# Initialize model

model = SimpleModel(num_classes=2, conv1_kernel_size=5,dropout=0.1)  # Adjust num_classes as per your requirement

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from PIL import Image

# Train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images = images.to(device)  
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Loss: 0.9612
Epoch [2/5], Loss: 0.7425
Epoch [3/5], Loss: 0.7197
Epoch [4/5], Loss: 0.7064
Epoch [5/5], Loss: 0.6954


In [ ]:
# Model evaluation
def evaluate_model(model, val_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    print(f"Validation Accuracy: {val_accuracy:.4f}")


evaluate_model(model, val_loader)

Validation Accuracy: 0.5150


## Model training and evaluation with hyperparameter tuning

In [ ]:
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

# Define the Skorch classifier with the given PyTorch model
net = NeuralNetClassifier(
    module=SimpleModel,
    module__num_classes=num_classes,  # Pass additional parameters to the model
    criterion=nn.CrossEntropyLoss,
    optimizer=optim.Adam,
    optimizer__lr=0.001,
    batch_size=32,
    max_epochs=5,
    device='cuda' if torch.cuda.is_available() else 'cpu',
)

# Extract data and labels from the DataLoader
X_train = []
y_train = []
for images, labels in train_loader:
    X_train.append(images.numpy())  # Assuming images are numpy arrays
    y_train.append(labels.numpy())  # Assuming labels are numpy arrays
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Define hyperparameters to tune
params = {
    'module__dropout': [0.1, 0.4, 0.6],
    'module__conv1_kernel_size': [2,3,4],

}

# Perform grid search with cross-validation
gs = GridSearchCV(net, params, cv=3, scoring='accuracy', verbose=1)

# Train the model with hyperparameter tuning
gs.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", gs.best_params_)
print("Best accuracy found: ", gs.best_score_)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.4032       0.9252        0.2122  14.1114
      2        0.1976       0.9252        0.2039  14.3127
      3        0.2001       0.9439        0.1539  15.2035
      4        0.1490       0.9626        0.1454  15.9099
      5        0.1455       0.9626        0.1145  16.1765
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.9637       0.9346        0.1862  16.4282
      2        0.2570       0.9813        0.1127  16.9342
      3        0.2168       0.9813        0.1081  15.4302
      4        0.1872       0.9907        0.0771  16.3055
      5        0.1598       0.9907        0.0679  16.5202
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1     

In [ ]:
best_net = gs.best_estimator_

scores=[]
with torch.no_grad():

    for images, labels in val_loader:
        score= best_net.score(images,labels)
        scores.append(score)
print(f"Validation Accuracy after Hyper parameter tuning: {np.mean(scores)}")

Validation Accuracy after Hyper parameter tuning: 0.9732142857142857
